In [1]:
import numpy as np
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.metrics import classification_report
import warnings
warnings.filterwarnings('ignore')

In [2]:
# Step 1: Create an imbalanced binary classification dataset
X, y = make_classification(n_samples=1000, n_features=10, n_informative=2, n_redundant=8, 
                           weights=[0.9, 0.1], flip_y=0, random_state=42)

print(np.unique(y, return_counts=True))

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, stratify=y, random_state=42)

(array([0, 1]), array([900, 100]))


### Experiment 1: Train Logistic Regression Classifier

In [3]:
log_reg = LogisticRegression(C=1, solver='liblinear')
log_reg.fit(X_train, y_train)
y_pred_log_reg = log_reg.predict(X_test)
print(classification_report(y_test, y_pred_log_reg))

              precision    recall  f1-score   support

           0       0.95      0.96      0.95       270
           1       0.60      0.50      0.55        30

    accuracy                           0.92       300
   macro avg       0.77      0.73      0.75       300
weighted avg       0.91      0.92      0.91       300



### Experiment 2: Train Random Forest Classifier

In [4]:
rf_clf = RandomForestClassifier(n_estimators=30, max_depth=3)
rf_clf.fit(X_train, y_train)
y_pred_rf = rf_clf.predict(X_test)
print(classification_report(y_test, y_pred_rf))

              precision    recall  f1-score   support

           0       0.96      1.00      0.98       270
           1       0.95      0.67      0.78        30

    accuracy                           0.96       300
   macro avg       0.96      0.83      0.88       300
weighted avg       0.96      0.96      0.96       300



### Experiment 3: Train XGBoost

In [5]:
xgb_clf = XGBClassifier(use_label_encoder=False, eval_metric='logloss')
xgb_clf.fit(X_train, y_train)
y_pred_xgb = xgb_clf.predict(X_test)
print(classification_report(y_test, y_pred_xgb))

              precision    recall  f1-score   support

           0       0.98      1.00      0.99       270
           1       0.96      0.80      0.87        30

    accuracy                           0.98       300
   macro avg       0.97      0.90      0.93       300
weighted avg       0.98      0.98      0.98       300



### Experiment 4: Handle class imbalance using SMOTETomek and then Train XGBoost

In [6]:
from imblearn.combine import SMOTETomek

smt = SMOTETomek(random_state=42)
X_train_res, y_train_res = smt.fit_resample(X_train, y_train)

print(np.unique(y_train_res, return_counts=True))

xgb_clf = XGBClassifier(use_label_encoder=False, eval_metric='logloss')
xgb_clf.fit(X_train_res, y_train_res)
y_pred_xgb = xgb_clf.predict(X_test)

print(classification_report(y_test, y_pred_xgb))

(array([0, 1]), array([619, 619]))
              precision    recall  f1-score   support

           0       0.98      0.98      0.98       270
           1       0.81      0.83      0.82        30

    accuracy                           0.96       300
   macro avg       0.89      0.91      0.90       300
weighted avg       0.96      0.96      0.96       300



#### Track Experiments using MLflow

In [7]:
models = [
    (
        "Logistic Regression", 
        {"C": 1, "solver": 'liblinear'},
        LogisticRegression(), 
        (X_train, y_train),
        (X_test, y_test)
    ),
    (
        "Random Forest", 
        {"n_estimators": 30, "max_depth": 3},
        RandomForestClassifier(), 
        (X_train, y_train),
        (X_test, y_test)
    ),
    (
        "XGBClassifier",
        {"use_label_encoder": False, "eval_metric": 'logloss'},
        XGBClassifier(), 
        (X_train, y_train),
        (X_test, y_test)
    ),
    (
        "XGBClassifier With SMOTE",
        {"use_label_encoder": False, "eval_metric": 'logloss'},
        XGBClassifier(), 
        (X_train_res, y_train_res),
        (X_test, y_test)
    )
]

In [8]:
reports = []

for model_name, params, model, train_set, test_set in models:
    X_train = train_set[0]
    y_train = train_set[1]
    X_test = test_set[0]
    y_test = test_set[1]
    
    model.set_params(**params)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    report = classification_report(y_test, y_pred, output_dict=True)
    reports.append(report)

In [9]:
import mlflow
import mlflow.sklearn
import mlflow.xgboost

#### dagshub setup

In [18]:
import dagshub
import mlflow
import os
import joblib

# Initialize DagsHub repo + MLflow
dagshub.init(repo_owner='srikanth.tadepalli208', 
             repo_name='Simple-implementation-of-MLflow', 
             mlflow=True)

os.environ['MLFLOW_TRACKING_USERNAME'] = 'srikanth.tadepalli208'
os.environ['MLFLOW_TRACKING_PASSWORD'] = '41157268c9ae821ff1c02d3d90e0b6071930d1ce'
os.environ['MLFLOW_TRACKING_URI'] = 'https://dagshub.com/srikanth.tadepalli208/Simple-implementation-of-MLflow.mlflow'

# Set experiment
mlflow.set_experiment("Anomaly Detection")

for i, element in enumerate(models):
    model_name = element[0]
    params = element[1]
    model = element[2]
    report = reports[i]
    
    with mlflow.start_run(run_name=model_name):
        # log params + metrics
        mlflow.log_params(params)
        mlflow.log_metrics({
            'accuracy': report['accuracy'],
            'recall_class_1': report['1']['recall'],
            'recall_class_0': report['0']['recall'],
            'f1_score_macro': report['macro avg']['f1-score']
        })

        # save model locally
        model_path = f"{model_name}_model.pkl"
        joblib.dump(model, model_path)

        # log model file as artifact
        mlflow.log_artifact(model_path, artifact_path="model")

        # (Optional) save input example for reference
        import pandas as pd
        pd.DataFrame(X_test).to_csv("input_example.csv", index=False)
        mlflow.log_artifact("input_example.csv", artifact_path="input_example")


Initialized MLflow to track repo "srikanth.tadepalli208/Simple-implementation-of-MLflow"

Repository srikanth.tadepalli208/Simple-implementation-of-MLflow initialized!

🏃 View run Logistic Regression at: https://dagshub.com/srikanth.tadepalli208/Simple-implementation-of-MLflow.mlflow/#/experiments/0/runs/6b3d586f39924309b160a2a5ab071ea8
🧪 View experiment at: https://dagshub.com/srikanth.tadepalli208/Simple-implementation-of-MLflow.mlflow/#/experiments/0
🏃 View run Random Forest at: https://dagshub.com/srikanth.tadepalli208/Simple-implementation-of-MLflow.mlflow/#/experiments/0/runs/a6639a70b120411195cd228d5941accd
🧪 View experiment at: https://dagshub.com/srikanth.tadepalli208/Simple-implementation-of-MLflow.mlflow/#/experiments/0
🏃 View run XGBClassifier at: https://dagshub.com/srikanth.tadepalli208/Simple-implementation-of-MLflow.mlflow/#/experiments/0/runs/9944c5c8e30a4daea650ae6582634906
🧪 View experiment at: https://dagshub.com/srikanth.tadepalli208/Simple-implementation-of-MLflow.mlflow/#/experiments/0
🏃 View run XGBClassifier With SMOTE at: https://dagshub.com/srikanth.tadepalli208/Simple-implementation-of-MLflow.mlflow/#/experiments/0/runs/e0f